## Data exploration

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image, ImageChops

In [ ]:
print(str(len(glob.glob("./training/11390/2012_01_05_17_06_01_0/*"))) + " Images for sample 1")
print(str(len(glob.glob("./training/*/*"))) + " Different Samples")
# Avg number of images per sample
print(str(len(glob.glob("./training/*/*/*"))/len(glob.glob("./training/*/*"))) + " Average number images per sample")

In [ ]:
# Data for a single sample
sample = []
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*"):
    print(i)
    sample.append(Image.open(i))

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_continuum.jpg"):
    print(i)

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_magnetogram.jpg"):
    print(i)

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_211.jpg"):
    print(i)

In [ ]:
# _# represents AIA wavelength for band #
# Hour Times: 05, 12, 15, 16
# Not sure what continuum images represent
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_304.jpg"):
    print(i)

In [ ]:
sample[14]

In [ ]:
img = sample[17]

In [ ]:
Image.open("./training/11390/2012_01_05_17_06_01_0/2012-01-05T153601__magnetogram.jpg")

In [ ]:
print(img.format)
print(img.mode)

In [ ]:
img

In [ ]:
#8E-07 is peak flux for this sample
# Data Transformation 1: 
i = img.split()[0]
len(i.histogram())

## Baseline network
This network should directly predict peak flux

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Adjusted convolutional layers for 3D data
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)

        # Adjusted pooling layers
        self.pool3d = nn.MaxPool3d(2, 2)
        
        self.fc1 = nn.Linear(64 * 32 * 32 * 1, 512)
        
        # Rest of the network remains the same
        self.fc2 = nn.Linear(512, 1)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # Add sequence of convolutional and max pooling layers
        x = self.pool3d(F.relu(self.conv1(x)))
        x = self.pool3d(F.relu(self.conv2(x)))
        x = self.pool3d(F.relu(self.conv3(x)))

        # Flatten image input
        x = x.view(-1, 64 * 32 * 32 * 1)  # Update these dimensions accordingly

        # Add dropout layer
        x = self.dropout(x)

        # Add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))

        # Add dropout layer
        x = self.dropout(x)

        # Add 2nd hidden layer for regression
        x = self.fc2(x)

        return x

# Instantiate the CNN for regression
model = SimpleCNN()
print(model)


SimpleCNN(
  (conv1): Conv3d(4, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3d): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


## Temperature Model
This model predicts temperature and then converts at the end to flux

In [3]:
class SimpleCNNWithWien(nn.Module):
    def __init__(self):
        super(SimpleCNNWithWien, self).__init__()
        # Adjusted convolutional layers for 3D data
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)

        # Adjusted pooling layers
        self.pool3d = nn.MaxPool3d(2, 2)
        
        # Linear layers
        # Update the input features of fc1 according to the output size of the last pooling layer
        # Example dimensions: 64 * 32 * 32 * 1 = 65536, if the final output size is [64, 32, 32, 1]
        self.fc1 = nn.Linear(64 * 32 * 32, 512)  # Adjust these dimensions
        self.fc2 = nn.Linear(512, 1)  # Outputting temperature
        self.dropout = nn.Dropout(0.25)

        # Wien's displacement constant (meters * Kelvin)
        self.wien_constant = 2.897771955e-3

    def forward(self, x):
        # Add sequence of convolutional and max pooling layers
        x = self.pool3d(F.relu(self.conv1(x)))
        x = self.pool3d(F.relu(self.conv2(x)))
        x = self.pool3d(F.relu(self.conv3(x)))

        # Flatten image input
        x = x.view(-1, 64 * 32 * 32)  # Update these dimensions accordingly
        x = self.dropout(x)

        # Add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Add 2nd hidden layer (outputs temperature)
        temperature = self.fc2(x)

        # Apply Wien's equation to calculate peak wavelength
        # Prevent division by zero in case temperature is zero
        temperature = torch.clamp(temperature, min=1e-6)
        peak_wavelength = self.wien_constant / temperature

        return peak_wavelength


## Data extraction
A single input will be a 4x256x256x10 matrix where dimensions represent: [timeinterval x height x width x wavelength/magnetogram]

In [11]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Get a list of all active region numbers
all_files = glob.glob("./training/*/*")

# Split all_files into 10 chunks
chunks = np.array_split(all_files, 40)
print(len(chunks[0]))

def generateChunk(idxs):
    wavelengths = ["94","131", "171","193","211","304","335","1700","continuum","magnetogram"]
    x = torch.zeros((1,4,256,256,10))
    y = torch.zeros((1,1))
    df = pd.read_csv('training/meta_data.csv')

    # First we will loop over every active region number
    train_files = chunks[0]
    for sample in tqdm(train_files):
        images = torch.empty((4,256,256,1), dtype=torch.int64)
        for wave in wavelengths:
            path = sample + "/*_{}.jpg".format(wave)
            pics = torch.tensor([np.array(Image.open(i)) for i in glob.glob(path)])
            for _ in range(4 - len(pics)):
                pics = torch.cat((pics, torch.zeros((1, 256,256))), 0)
            pics = pics.reshape(4,256,256,1)
            images = torch.cat((images, pics), 3)
        images = images[:,:,:,1:]
        images = images.reshape(1,4,256,256,10)
        x = torch.cat((x, images), 0)
        idx = path.split("/")[2] + "_"+ path.split("/")[3]
        y = torch.cat((y, torch.tensor(df[df["id"] == idx]['peak_flux'].iloc[0]).reshape(-1,1)), 0)
    return x, y

#for i in range(40):
#    train_x, train_y = generateChunk(chunks[i])
#    dataset = TensorDataset(train_x, train_y)
#    torch.save(dataset, "datachunks/chunk_{}.pt".format(i))

## Create a DataLoader
#val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

209


In [13]:
def train_model(model, criterion, optimizer, epochs):
    for epoch in range(epochs):
        for i in tqdm(range(39)):
            train_x, train_y = generateChunk(chunks[i])
            dataset = TensorDataset(train_x, train_y)
            train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
            model.train()  # Set the model to training mode
            for inputs, targets in train_loader:
                optimizer.zero_grad()  # Reset the gradients
                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets.float())  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update weights

        train_x, train_y = generateChunk(chunks[39])
        dataset = TensorDataset(train_x, train_y)
        val_loader = DataLoader(dataset, batch_size=32, shuffle=True)
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # No need to track gradients in validation phase
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, acc: {torch.sum(outputs == targets) / len(targets)}')

mod = SimpleCNN()
train_model(mod, nn.MSELoss(), torch.optim.Adam(mod.parameters()), 10)
mod = SimpleCNNWithWien()
train_model(mod, nn.MSELoss(), torch.optim.Adam(mod.parameters()), 10)

  3%|▎         | 1/39 [05:00<3:10:04, 300.11s/it]